In [7]:
import torch #to import pytorch
import torch.nn as nn #to import the neural network library as nn
import torch.optim as optim #to import the optimisation library as optim
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset # data loader importing
import os

In [5]:
# Load the dataset
path = os.path.join("/kaggle/input/concretedataset/Concrete_Data.csv")
concrete_data = pd.read_csv(path)

In [6]:
concrete_data

,cement,slag,flyash,water,superplas,coarse_agg,fine_agg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [9]:
# Separate features and target
X = concrete_data.drop(columns=['strength'])
y = concrete_data[['strength']]

In [13]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
X_train

array([[-1.16087734,  0.85738747,  0.9824489 , ..., -0.25746155,
        -0.64752011, -0.27567315],
       [ 1.30862304, -0.60249189,  1.23259821, ..., -1.92694961,
        -0.2731482 , -0.27567315],
       [-0.0768653 , -0.85558366,  1.06687429, ...,  1.01785948,
         0.06662828, -0.68931339],
       ...,
       [-0.86591441, -0.85558366,  1.12628475, ...,  1.34082214,
         0.33103616,  0.91561074],
       [ 1.78316909,  0.51111191, -0.83113361, ..., -1.54422615,
         0.11605031, -0.27567315],
       [ 0.28509237, -0.85558366,  0.9355459 , ..., -0.6172979 ,
         0.13458358, -0.27567315]])

In [42]:
# Convert data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [43]:
X_train_tensor.shape

torch.Size([824, 8])

In [53]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [54]:
# Define the ANN model
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.input_layer = nn.Linear(input_dim, 64)
        self.hidden_layer = nn.Linear(64, 32)
        self.hidden_layer2 = nn.Linear(32, 8)
        self.output_layer = nn.Linear(8, 1)  # Output layer with 1 neurons for 1 outputs

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        x = torch.relu(self.hidden_layer(x))
        x = torch.relu(self.hidden_layer2(x))
        x = self.output_layer(x)
        return x

In [55]:
# Initialize the model
input_dim = X_train.shape[1]
model = ANN(input_dim)

In [56]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}')

Epoch [1/500], Loss: 1515.3446044921875
Epoch [2/500], Loss: 743.1970930833083
Epoch [3/500], Loss: 185.35461953970102
Epoch [4/500], Loss: 139.2203914935772
Epoch [5/500], Loss: 132.0504860511193
Epoch [6/500], Loss: 117.04020045353816
Epoch [7/500], Loss: 100.02680793175331
Epoch [8/500], Loss: 98.4258058988131
Epoch [9/500], Loss: 141.41987492487982
Epoch [10/500], Loss: 79.03017513568585
Epoch [11/500], Loss: 93.36850533118614
Epoch [12/500], Loss: 85.49883915827824
Epoch [13/500], Loss: 97.40493803757887
Epoch [14/500], Loss: 96.53049615713266
Epoch [15/500], Loss: 93.54944581251878
Epoch [16/500], Loss: 57.25856927724985
Epoch [17/500], Loss: 59.56708497267503
Epoch [18/500], Loss: 58.430218329796425
Epoch [19/500], Loss: 101.81065221933218
Epoch [20/500], Loss: 93.46622789823093
Epoch [21/500], Loss: 53.95529556274414
Epoch [22/500], Loss: 48.20502589299129
Epoch [23/500], Loss: 90.46743158193735
Epoch [24/500], Loss: 52.975609852717476
Epoch [25/500], Loss: 44.24969438406137
Ep

In [60]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001) #reducing the learning rate even more to get even better result

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}')

Epoch [1/100], Loss: 4.776674032211304
Epoch [2/100], Loss: 4.739891253984892
Epoch [3/100], Loss: 4.7446750310751105
Epoch [4/100], Loss: 4.75280028123122
Epoch [5/100], Loss: 4.758197344266451
Epoch [6/100], Loss: 4.7244895329842205
Epoch [7/100], Loss: 4.74842014679542
Epoch [8/100], Loss: 4.728320910380437
Epoch [9/100], Loss: 4.777014860740075
Epoch [10/100], Loss: 4.766154802762545
Epoch [11/100], Loss: 4.753142411892231
Epoch [12/100], Loss: 4.778932828169602
Epoch [13/100], Loss: 4.744372844696045
Epoch [14/100], Loss: 4.749992792422955
Epoch [15/100], Loss: 4.764149335714487
Epoch [16/100], Loss: 4.762575387954712
Epoch [17/100], Loss: 4.73381586258228
Epoch [18/100], Loss: 4.768836388221154
Epoch [19/100], Loss: 4.723276230005117
Epoch [20/100], Loss: 4.799048203688401
Epoch [21/100], Loss: 4.747911957594065
Epoch [22/100], Loss: 4.722835485751812
Epoch [23/100], Loss: 4.82063075212332
Epoch [24/100], Loss: 4.762863360918486
Epoch [25/100], Loss: 4.748732236715464
Epoch [26/1

In [62]:
# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')

Test Loss: 29.01835060119629
